In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install langchain

In [3]:
from langchain import HuggingFaceHub
import os
from dotenv import load_dotenv
import pdfplumber

In [4]:
from langchain_text_splitters import CharacterTextSplitter

def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text() + '\n'
    return text

def save_chunks_to_txt(chunks, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for i, chunk in enumerate(chunks):
            f.write(f"Chunk {i+1}:\n")
            f.write(chunk)
            f.write("\n" + "-"*40 + "\n")  

file_path = 'ArtificialIntelligenceAct-1-50.pdf'  
pdf_text = read_pdf(file_path)

# testar valores e splitters e no final ver quais os melhores
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(pdf_text)
output_file = 'chunks.txt'

save_chunks_to_txt(chunks, output_file)

In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(chunks)
print(len(embeddings[0]))

384


- Armazenar embeddings numa DB vetorial p ex qdrant, postgreSQL

In [20]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# UI : http://localhost:6333/dashboard
client = QdrantClient(url="http://localhost:6333")


client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [21]:
from qdrant_client.models import PointStruct

points = [
    PointStruct(id=i, vector=embedding.tolist(), payload={"text": chunk})
    for i, (embedding, chunk) in enumerate(zip(embeddings, chunks))
]

operation_info = client.upsert(
    collection_name="test_collection",
    points=points,
    wait=True
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [27]:
ex = "What is Artificial Intelligence Act?"
query = model.encode(ex)
print(len(query))

384


In [32]:
"""search_result = client.query_points(
    collection_name="test_collection",
    query=query,
    with_payload=False,
    limit=3
).points

print(search_result)"""
pergunta = "How does the Artificial Intelligence Act address the potential risks posed by AI systems to fundamental rights and public safety in the EU?"
query_vector = model.encode([pergunta])[0]  


search_result = client.search(
    collection_name="test_collection",  
    query_vector=query_vector.tolist(),  
    limit=1 
)


print("Results:")
for hit in search_result:
    print(f"Resultado ID: {hit.id}, Texto: {hit.payload['text']}")

Results:
Resultado ID: 10, Texto: and level of technological development, AI may generate risks and cause harm to public
interests and fundamental rights that are protected by Union law. Such harm might be
material or immaterial, including physical, psychological, societal or economic harm.
(6) Given the major impact that AI can have on society and the need to build trust, it is vital
for AI and its regulatory framework to be developed in accordance with Union values as
enshrined in Article 2 of the Treaty on European Union (TEU), the fundamental rights
and freedoms enshrined in the Treaties and, pursuant to Article 6 TEU, the Charter. As
a pre-requisite, AI should be a human-centric technology. It should serve as a tool for
people, with the ultimate aim of increasing human well-being.
(7) In order to ensure a consistent and high level of protection of public interests as regards
health, safety and fundamental rights, common rules for high-risk AI systems should be
